In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
import math
from torch.cuda.amp import autocast, GradScaler
import random

In [5]:
# ------------------------------------------------------------------------
# 1) A discrete and Sigmoid diffusion schedule
# ------------------------------------------------------------------------
class DiscreteDiffusionSchedule:
    """
    Simple linear schedule of alpha_t from t=1..T,
    where alpha_t = min_alpha + (max_alpha - min_alpha)*(t/T).
    """
    def __init__(self, T=10, min_alpha=0.1, max_alpha=0.7):
        self.T = T
        self.alphas = []
        for t in range(1, T+1):
            frac = t / T
            alpha_t = min_alpha + (max_alpha - min_alpha)*frac
            self.alphas.append(alpha_t)

    def __len__(self):
        return self.T

    def __getitem__(self, t):
        # t in [1..T], python indexing 0..T-1
        return self.alphas[t-1]

class SigmoidDiffusionSchedule:
    """
    Sigmoid schedule of alpha_t from t=1..T.

    alpha_t = min_alpha + (max_alpha - min_alpha)*sigmoid(k*(frac - 0.5)),
    where frac = (t-1)/(T-1).
    """
    def __init__(self, T=30, min_alpha=0.1, max_alpha=0.7, k=12.0):
        self.T = T
        self.alphas = []
        for t in range(1, T+1):
            # frac in [0..1]
            frac = (t - 1) / (T - 1)  
            # logistic
            s = 1 / (1 + math.exp(-k * (frac - 0.5)))
            alpha_t = min_alpha + (max_alpha - min_alpha) * s
            self.alphas.append(alpha_t)

    def __len__(self):
        return self.T

    def __getitem__(self, t):
        # t in [1..T], python indexing 0..T-1
        return self.alphas[t - 1]

class OneFlipDiffusionSchedule:
    """
    Diffusion schedule where `t` digits are flipped at step `t`.
    """
    def __init__(self, T=50, puzzle_size=81):
        """
        Args:
            T (int): Number of diffusion steps.
            puzzle_size (int): Total number of digits in the puzzle (e.g., 81 for 9x9 Sudoku).
        """
        self.T = T
        self.puzzle_size = puzzle_size
        self.flip_counts = [min(t, puzzle_size) for t in range(1, T + 1)]  # `t` flips at step `t`

    def __len__(self):
        return self.T

    def __getitem__(self, t):
        """
        Returns the number of flips to apply at step t.
        Args:
            t (int): Step index (1-based, i.e., t in [1..T]).
        """
        return self.flip_counts[t - 1]



In [40]:
# ------------------------------------------------------------------------
# 2) Forward noising that respects puzzle givens
# ------------------------------------------------------------------------
def forward_diffusion_with_puzzle(puzzle, solution, t, schedule, vocab_size, device):
    """
    puzzle:   (batch, 81) with digits in [0..9]. 0 means blank, non-zero means given.
    solution: (batch, 81) correct final solution
    t:        an integer in [1..T]
    schedule: contains alpha_t
    returns x_t: partially noised solution (batch, 81)
        - givens remain the same as solution's corresponding digit
        - blank positions get replaced with random digits w.p. alpha_t
    """
    alpha_t = schedule[t]  # fraction to noise
    puzzle = puzzle.to(device)
    solution = solution.to(device)

    # Where puzzle is nonzero => givens => do NOT overwrite
    givens_mask = (puzzle != 0)

    # We'll noise only the positions that are blank in the puzzle
    #   i.e. puzzle[i] == 0 => we can noise solution[i].
    blank_mask = (puzzle == 0)

    # Create random noise from [0..vocab_size-1] for the blank positions
    noise = torch.randint(0, vocab_size, solution.shape, device=device)

    # Decide which blank positions to replace with noise
    replace_mask = (torch.rand_like(solution.float()) < alpha_t) & blank_mask

    # x_t: start from the true solution, then replace with noise for some blank cells
    x_t = solution.clone()
    x_t[replace_mask] = noise[replace_mask]

    # givens remain the same as the correct solution digit at that position
    # (actually, this is already the default if puzzle != 0, but we do not overwrite them)
    # so x_t[givens_mask] = solution[givens_mask] # if you want to be explicit

    return x_t

def forward_diffusion_mixed(puzzle, solution, t, schedule, vocab_size, device, zero_bias=0.8, bias_increment=1.3):
    """
    Hybrid diffusion: flips `t` digits (1-flip pattern) with a bias toward flipping to zeros.
    The zero_bias increases multiplicatively after each random flip to limit multiple wrong positions.

    Args:
        puzzle: (batch, 81) Original puzzle with digits in [0..9], where 0 means blank.
        solution: (batch, 81) Correct final solution.
        t: Step in the schedule (1..T).
        schedule: Diffusion schedule determining the number of flips.
        vocab_size: Total number of possible values (e.g., 10 for digits 0-9).
        device: Torch device (e.g., "cuda").
        zero_bias: Initial probability of flipping to zero instead of a random digit.
        bias_increment: Multiplicative factor to increase zero_bias (default: 1.2).

    Returns:
        x_t: Noised solution (batch, 81).
    """
    puzzle = puzzle.to(device)
    solution = solution.to(device)

    # Number of flips determined by the schedule (1-flip pattern)
    num_flips = int(schedule[t])
    batch_size, puzzle_size = solution.shape
    x_t = solution.clone()

    for b in range(batch_size):
        # Select `num_flips` random positions to flip
        all_positions = torch.arange(puzzle_size, device=device)
        flip_indices = torch.randperm(puzzle_size, device=device)[:num_flips]

        current_zero_bias = zero_bias  # Start with the initial zero_bias for this batch

        for idx in flip_indices:
            if puzzle[b, idx] != 0:
                # Given digits: Flip with a small probability
                if random.random() > 0.99:  # Flip only 1% of the time
                    x_t[b, idx] = random.randint(1, vocab_size - 1)
            else:
                # Blank cells: Flip with bias toward zero
                if random.random() < current_zero_bias:
                    x_t[b, idx] = 0  # Flip to zero
                else:
                    x_t[b, idx] = random.randint(1, vocab_size - 1)  # Flip to random digit
                    # Increment the bias multiplicatively
                    current_zero_bias = min(1.0, current_zero_bias * bias_increment)

    return x_t

In [30]:
# ------------------------------------------------------------------------
# 3) The model sees puzzle+partially noised solution as input
# ------------------------------------------------------------------------
class PuzzleDenoiser(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, max_T=40):
        super().__init__()
        self.vocab_size = vocab_size
        
        # Standard embeddings
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 1) Time embedding for t in [1..max_T]
        self.time_embedding = nn.Embedding(max_T + 1, embed_dim)
        
        # Positional embedding for puzzle+solution sequence (learned initialization)
        self.pos_embedding = nn.Parameter(torch.zeros(1, 163, embed_dim))
        nn.init.normal_(self.pos_embedding, mean=0.0, std=0.02)  # Custom initialization
        
        # Layer norm after positional embeddings
        self.post_pos_norm = nn.LayerNorm(embed_dim)
        
        # Transformer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output layer
        self.output_layer = nn.Linear(embed_dim, vocab_size)

    def forward(self, puzzle, x_t, t):
        """
        puzzle, x_t: (batch, 81)
        t: (batch,) integer steps in [1..max_T], describing which noising step
        """
        batch_size = puzzle.size(0)
        
        # Concatenate [puzzle || x_t] => shape (batch, 162)
        inp = torch.cat([puzzle, x_t], dim=1)  # (batch, 162)
        
        # Token embeddings for puzzle + x_t
        emb = self.embedding(inp)  # => (batch, 162, embed_dim)
        
        # 2) Time embedding => Broadcast time embedding across all tokens
        t_emb = self.time_embedding(t).unsqueeze(1)  # (batch, 1, embed_dim)
        t_broadcast = t_emb.expand(-1, 162, -1)  # Broadcast to match sequence length
        emb = emb + t_broadcast
        
        # 3) Add positional embeddings
        cat_emb = emb + self.pos_embedding[:, :162, :]  # Slice positional embeddings to match sequence length
        
        # Apply layer normalization after positional embeddings
        cat_emb = self.post_pos_norm(cat_emb)
        
        # Pass through Transformer
        enc_out = self.encoder(cat_emb)  # => (batch, 162, embed_dim)
        
        # Apply LayerNorm after Transformer (optional)
        enc_out = self.post_pos_norm(enc_out)
        
        # Output layer => (batch, 162, vocab_size)
        logits = self.output_layer(enc_out)
        
        return logits



In [47]:
# ------------------------------------------------------------------------
# 4) Diffusion train step
# ------------------------------------------------------------------------
def compute_subgoal_weights(puzzle, solution, vocab_size=10):
    """
    Computes a difficulty-based weight for each cell in the puzzle:
      - If puzzle[i] != 0, it's a given => typically weight = 0 or minimal (since it's known).
      - If puzzle[i] == 0, compute how many digits (1..9) are valid in that cell
        given Sudoku constraints. The more valid candidates => the harder the subgoal => higher weight.

    Returns:
        weights: A tensor of shape (batch, 81) with floating-point weights.
    """

    batch_size = puzzle.size(0)
    weights = torch.zeros_like(solution, dtype=torch.float32)

    # We'll iterate over each puzzle in the batch
    for b in range(batch_size):
        # puzzle[b]: shape (81,)
        # solution[b]: shape (81,)

        # Convert puzzle[b] into a 9x9 grid for easier row/col/box indexing
        puzzle_grid = puzzle[b].view(9, 9).cpu().numpy()  # shape (9,9), on CPU
        # We'll also want to compute constraints for each row, col, box
        # but let's do it cell-by-cell.

        for idx in range(81):
            r = idx // 9  # row
            c = idx % 9   # column

            if puzzle_grid[r, c] != 0:
                # It's a given => we can optionally set weight to 0 or a small value
                weights[b, idx] = 0.0
            else:
                # It's blank => compute how many valid digits remain
                row_vals = set(puzzle_grid[r, :].tolist())
                col_vals = set(puzzle_grid[:, c].tolist())

                # Identify which 3x3 box (by top-left corner)
                box_row = (r // 3) * 3
                box_col = (c // 3) * 3
                box_vals = set(
                    puzzle_grid[box_row:box_row+3, box_col:box_col+3].reshape(-1).tolist()
                )

                # Givens can be 1..9, ignoring 0 (blank)
                used_vals = (row_vals | col_vals | box_vals) - {0}
                # valid digits are those in [1..9] not in used_vals
                all_digits = set(range(1, vocab_size))  # {1,2,...,9} if vocab=10
                valid_candidates = all_digits - used_vals

                num_candidates = len(valid_candidates)

                # Weight logic:
                # e.g., let weight = num_candidates
                # or weight = 1 + num_candidates, or scale by some factor
                # The bigger the number of candidates => the bigger the weight
                weights[b, idx] = float(num_candidates)

    # Optionally normalize weights per puzzle
    weights = weights / (weights.max(dim=1, keepdim=True)[0].clamp(min=1.0) + 1e-8)

    return weights

def diffusion_train_step(
    model,
    puzzle,
    solution,
    schedule,
    optimizer,
    vocab_size,
    device,
    loss_fn,
    zero_bias=0.8,
    use_weighted_loss=False
):
    # Ensure input tensors are on the correct device and have the right type
    puzzle = puzzle.to(dtype=torch.long, device=device)
    solution = solution.to(dtype=torch.long, device=device)

    # Get a random diffusion step
    T = len(schedule)
    t_int = np.random.randint(1, T + 1)
    t_tensor = torch.tensor([t_int] * puzzle.size(0), device=device)

    # Generate noisy puzzle (x_t)
    x_t = forward_diffusion_mixed(puzzle, solution, t_int, schedule, vocab_size, device, zero_bias=zero_bias)

    # Forward pass through the model
    with torch.cuda.amp.autocast():  # Ensure everything is within autocast
        logits = model(puzzle, x_t, t_tensor)  # Shape: (batch, 163, vocab_size)

        # Extract logits for the solution part
        logits_solution_part = logits[:, 82:, :]  # Shape: (batch, 81, vocab_size)

        # Compute cross-entropy loss
        ce_loss = loss_fn(
            logits_solution_part.reshape(-1, vocab_size),
            solution.reshape(-1)
        )

        if use_weighted_loss:
            # Compute subgoal weights if enabled
            weights = compute_subgoal_weights(puzzle, solution, vocab_size).to(device)
            weights_flat = weights.reshape(-1)

            # Compute per-token loss (NLL)
            nll_per_token = torch.nn.functional.cross_entropy(
                logits_solution_part.reshape(-1, vocab_size),
                solution.reshape(-1),
                reduction='none'
            )

            # Apply subgoal weights
            weighted_nll = nll_per_token * weights_flat

            # Compute final weighted loss
            loss = weighted_nll.mean()
        else:
            loss = ce_loss

    return loss



In [34]:
# ------------------------------------------------------------------------
# 5) Validation step
# ------------------------------------------------------------------------
@torch.no_grad()
def diffusion_eval_step(
    model, 
    puzzle,
    solution,
    schedule,
    loss_fn,
    vocab_size,
    device
):
    """
    Performs a forward pass during evaluation with time conditioning.

    Args:
        model: The PuzzleDenoiser model with time conditioning.
        puzzle: Tensor of shape (batch, 81) with puzzle digits (0 for blanks).
        solution: Tensor of shape (batch, 81) with solution digits.
        schedule: The diffusion schedule object containing alphas and T.
        loss_fn: The loss function, e.g., nn.CrossEntropyLoss().
        vocab_size: Size of the vocabulary (digits 0-9 => 10).
        device: torch.device to perform computations on.

    Returns:
        loss: The evaluation loss as a float.
    """
    model.eval()
    puzzle = puzzle.to(device)
    solution = solution.to(device)

    # 1. Sample a random diffusion step t for the entire batch
    T = schedule.T
    t_int = np.random.randint(1, T + 1)  # Sample t in [1, T]
    # Create a tensor of shape (batch_size,) filled with t_int
    t_tensor = torch.full((puzzle.size(0),), t_int, dtype=torch.long, device=device)

    # 2. Create x_t with forward diffusion
    x_t = forward_diffusion_with_puzzle(
        puzzle, solution, t_int, schedule, vocab_size, device
    )

    # 3. Forward pass with time conditioning
    logits = model(puzzle, x_t, t_tensor)  # shape: (batch, 162, vocab_size)

    # 4. Slice out the solution part
    logits_solution_part = logits[:, 81:, :]  # Include only solution tokens

    # 5. Compute cross-entropy loss
    loss = loss_fn(
        logits_solution_part.reshape(-1, vocab_size),  # (batch*81, vocab_size)
        solution.reshape(-1)                           # (batch*81,)
    )
    
    return loss.item()

In [35]:
# ------------------------------------------------------------------------
# 6) Iterative decoding to fill blank cells
# ------------------------------------------------------------------------
@torch.no_grad()
def iterative_decode(
    model,
    puzzle,        # shape (batch, 81) puzzle givens
    schedule,
    vocab_size,
    device,
    steps=None
):
    """
    Iteratively decodes the puzzle by denoising from t=T to t=1 with time conditioning.

    Args:
        model: The trained diffusion model.
        puzzle: (batch, 81) Input puzzle with given digits and blanks.
        schedule: The diffusion schedule object.
        vocab_size: Total vocabulary size (e.g., 10 for digits 0-9).
        device: Torch device for evaluation (e.g., 'cuda').
        steps: Optional; number of decoding steps (defaults to schedule.T).

    Returns:
        x_t: The final decoded solution (batch, 81).
    """
    model.eval()
    puzzle = puzzle.to(device)
    batch_size, seq_len = puzzle.shape
    T = schedule.T if steps is None else steps

    # Initialize x_t: noise only in blank positions
    x_t = puzzle.clone()
    blank_mask = (puzzle == 0)
    x_t[blank_mask] = torch.randint(0, vocab_size, x_t[blank_mask].shape, device=device)

    for curr_t in range(T, 0, -1):
        # Create a tensor of the current step, repeated for the batch
        t_tensor = torch.full((batch_size,), curr_t, dtype=torch.long, device=device)

        # Forward pass with time conditioning
        logits = model(puzzle, x_t, t_tensor)  # shape: (batch, 162, vocab_size)

        # Slice out the solution part
        logits_solution_part = logits[:, 81:, :]  # Include only solution tokens

        # Predict the solution tokens
        pred_sol = logits_solution_part.argmax(dim=-1)  # shape: (batch, 81)

        # Update only blank cells
        x_t[blank_mask] = pred_sol[blank_mask]

    return x_t  # Final filled solution

In [36]:
@torch.no_grad()
def validate_combined(
    model, 
    schedule,
    loader,        # A DataLoader for validation
    device,
    loss_fn,
    vocab_size=10
):
    """
    Combines validation loss, solve rate, and token accuracy in one pass.
    """
    model.eval()

    total_loss = 0.0
    total_samples = 0
    solved_count = 0
    correct_token_count = 0
    total_token_count = 0

    with torch.cuda.amp.autocast():
        for puzzles, solutions in tqdm(loader, desc="Validation"):
            puzzles = puzzles.to(device)
            solutions = solutions.to(device)
            batch_size = puzzles.size(0)

            # 1) Compute a random t for the entire batch and get x_t
            t_int = np.random.randint(1, schedule.T + 1)
            t_tensor = torch.full((batch_size,), t_int, dtype=torch.long, device=device)

            x_t = forward_diffusion_with_puzzle(
                puzzle=puzzles,
                solution=solutions,
                t=t_int,
                schedule=schedule,
                vocab_size=vocab_size,
                device=device
            )
            
            # 2) Forward pass with time conditioning
            logits = model(puzzles, x_t, t_tensor)                  # shape (batch, 162, vocab_size)
            logits_solution_part = logits[:, 81:, :]                # only solution part
            
            # 3) Compute cross-entropy loss
            loss = loss_fn(
                logits_solution_part.reshape(-1, vocab_size),
                solutions.reshape(-1)
            )
            
            total_loss += loss.item() * batch_size
            total_samples += batch_size

            # 4) Iterative decode to get final filled boards
            x_filled = iterative_decode(
                model=model,
                puzzle=puzzles,
                schedule=schedule,
                vocab_size=vocab_size,
                device=device
            )
            
            # 5) Solve rate: how many boards are 100% correct
            eq_mask = (x_filled == solutions)          # (batch, 81)
            batch_solved = eq_mask.all(dim=1).sum().item()
            solved_count += batch_solved

            # 6) Token accuracy on masked cells
            masked_mask = (puzzles == 0)               # (batch, 81)
            correct_masked = eq_mask & masked_mask      # Correct predictions on masked cells
            correct_token_count += correct_masked.sum().item()
            total_token_count += masked_mask.sum().item()

    avg_loss = total_loss / total_samples
    solve_rate = solved_count / total_samples
    token_acc = correct_token_count / total_token_count if total_token_count > 0 else 0.0

    return avg_loss, solve_rate, token_acc


In [43]:
# ------------------------------------------------------------------------
# 7) Putting it all together: example training loop
# ------------------------------------------------------------------------
def train_puzzle_diffusion(
    X_train, y_train, 
    X_val,   y_val,
    vocab_size=10,
    T=40,
    embed_dim=512,
    num_heads=8,
    num_layers=8,
    batch_size=64,
    num_epochs=50,
    best_model_path=None
):
    """
    Trains the PuzzleDenoiser model with time-conditioned diffusion.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    schedule = SigmoidDiffusionSchedule(
        T=T, 
        min_alpha=0.02, 
        max_alpha=0.7,
        k=12.0
    )

    # Handle multi-GPU setup
    multi_gpu = torch.cuda.device_count() > 1
    if multi_gpu:
        print(f"Using {torch.cuda.device_count()} GPUs!")
    else:
        print("Using 1 GPU or CPU.")

    # Initialize the model with max_T matching the schedule
    model = PuzzleDenoiser(
        vocab_size=vocab_size, 
        embed_dim=embed_dim, 
        num_heads=num_heads, 
        num_layers=num_layers,
        max_T=T
    ).to(device)

    # Load best model if provided (handling DataParallel)
    if best_model_path is not None:
        state_dict = torch.load(best_model_path, map_location=device)
        
        if multi_gpu and not list(state_dict.keys())[0].startswith('module.'):
            model.load_state_dict(state_dict)
            model = nn.DataParallel(model)
        elif not multi_gpu and list(state_dict.keys())[0].startswith('module.'):
            new_state_dict = {k[7:]: v for k, v in state_dict.items()}
            model.load_state_dict(new_state_dict)
        else:
            model.load_state_dict(state_dict)
            
        print(f"Loaded model from {best_model_path}")
    elif multi_gpu:
        model = nn.DataParallel(model)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    # Add learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3,
        verbose=True
    )
    loss_fn = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()

    # Setup data loaders
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset   = TensorDataset(X_val,   y_val)
    train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader    = DataLoader(val_dataset,   batch_size=batch_size)

    best_val_loss = float('inf')

    # Initial validation before training
    if False:
        print("Running initial validation (Epoch 0):")
        avg_val_loss, val_solve_rate, val_token_acc = validate_combined(
            model=model,
            schedule=schedule,
            loader=val_loader,
            device=device,
            loss_fn=loss_fn,
            vocab_size=vocab_size
        )
        print(f"Initial val_loss={avg_val_loss:.4f}")
        print(f"Initial solve rate on validation set: {val_solve_rate*100:.2f}%")
        print(f"Initial token-level accuracy on validation set: {val_token_acc*100:.2f}%")

    for epoch in range(num_epochs):
        model.train()
        train_losses = []
        for batch_puzzle, batch_solution in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
            # Use autocast for mixed precision training
            with torch.cuda.amp.autocast():
                loss_val = diffusion_train_step(
                    model=model,
                    puzzle=batch_puzzle,
                    solution=batch_solution,
                    schedule=schedule,
                    optimizer=optimizer,
                    loss_fn=loss_fn,
                    vocab_size=vocab_size,
                    device=device
                )
            
            # Scale loss and do backward pass
            scaler.scale(loss_val).backward()
            
            # Unscale gradients and optimizer step
            scaler.step(optimizer)
            scaler.update()
            
            # Zero gradients
            optimizer.zero_grad()
            
            train_losses.append(loss_val.item())
            
        avg_train_loss = np.mean(train_losses)

        # Validation
        avg_val_loss, val_solve_rate, val_token_acc = validate_combined(
            model=model,
            schedule=schedule,
            loader=val_loader,
            device=device,
            loss_fn=loss_fn,
            vocab_size=vocab_size
        )

        print(f"Epoch {epoch+1}: train_loss={avg_train_loss:.4f}, val_loss={avg_val_loss:.4f}")
        print(f"Solve rate on validation set: {val_solve_rate*100:.2f}%")
        print(f"Token-level accuracy on validation set: {val_token_acc*100:.2f}%")

        # Update learning rate scheduler
        scheduler.step(avg_val_loss)

        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "puzzle_diffuser_best.pt")
            print(f"  [*] Best model saved @ val_loss={avg_val_loss:.4f}")

    return model, schedule


In [7]:
# Load and preprocess data
import pandas as pd
from sklearn.model_selection import train_test_split

# Load sudoku data
df = pd.read_csv('./data/sudoku.csv')

# Convert strings to tensors
def preprocess_sudoku(puzzle_str):
    # Convert string to list of integers and then to tensor
    return torch.tensor([int(d) for d in puzzle_str], dtype=torch.long)

# Convert all puzzles and solutions
puzzles = torch.stack([preprocess_sudoku(p) for p in df['quizzes']])
solutions = torch.stack([preprocess_sudoku(s) for s in df['solutions']])

# Karpathy split (90/5/5)
train_size = 0.9
val_size = 0.05
test_size = 0.05

# First split into train and temp
X_train, X_temp, y_train, y_temp = train_test_split(
    puzzles, solutions, train_size=train_size, random_state=42
)

# Split temp into val and test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

In [ ]:
model, schedule = train_puzzle_diffusion(
    X_train, y_train,
    X_val,   y_val,
    vocab_size=10,
    T=40,
    embed_dim=512,
    num_heads=8,
    num_layers=8,
    batch_size=64,
    num_epochs=50
    #best_model_path="puzzle_diffuser_best.pt"
)

In [48]:
model, schedule = train_puzzle_diffusion(
    X_train, y_train,
    X_val,   y_val,
    vocab_size=10,
    T=80,
    embed_dim=256,
    num_heads=4,
    num_layers=4,
    batch_size=64,
    num_epochs=100
    #best_model_path="puzzle_diffuser_best.pt"
)

C:\Users\drob7\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\drob7\AppData\Local\Temp\ipykernel_33480\2404183206.py:72: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Using device: cuda
Using 2 GPUs!


Epoch 1 [Train]:   0%|          | 0/14063 [00:00<?, ?it/s]C:\Users\drob7\AppData\Local\Temp\ipykernel_33480\2404183206.py:102: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
C:\Users\drob7\AppData\Local\Temp\ipykernel_33480\3673127547.py:91: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Ensure everything is within autocast
Epoch 1 [Train]:   0%|          | 0/14063 [00:00<?, ?it/s]


ValueError: Expected input batch_size (5120) to match target batch_size (5184).

In [ ]:
#############################################
# 2) CREATE THE NEW SCHEDULE
#############################################
schedule = OneFlipDiffusionSchedule(
    T=80
)

#############################################
# 3) PICK A SAMPLE PUZZLE & SOLUTION
#############################################
sample_puzzle = X_train[0:1]
sample_solution = y_train[0:1]

print("Puzzle:", sample_puzzle)
print("Solution:", sample_solution)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#############################################
# 4) SHOW x_1, x_2, ... x_T
#############################################
vocab_size = 10  # digits 0-9
for t in range(1, schedule.T + 1):
    x_t = forward_diffusion_mixed(X_train[:1], y_train[:1], t, schedule, vocab_size=10, device=device, bias_increment=1.3)
    print(f"x_t at t={t}:", x_t)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Then to do iterative decoding:
puzzle_batch = X_val[0:2]  # for example
x_filled = iterative_decode(
    model, 
    puzzle=puzzle_batch, 
    schedule=schedule, 
    vocab_size=10,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)
print("Puzzle: ", puzzle_batch)
print("Decoded solution: ", x_filled)
print("Original solution: ", y_val[0:2])

NameError: name 'iterative_decode' is not defined

In [ ]:
accuracy = measure_sudoku_solve_rate(
    model,
    schedule,
    X_val[0:100],
    y_val[0:100],
    device=device,
    vocab_size=10,
    batch_size=32
)

print(f"Solve rate on validation set: {accuracy*100:.2f}%")

In [ ]:
# Create a small validation dataset for testing
X_val_test = X_val[:1000]
y_val_test = y_val[:1000]
test_val_loader = DataLoader(
    TensorDataset(X_val_test, y_val_test),
    batch_size=32,
    shuffle=False
)

# Initialize loss function
loss_fn = nn.CrossEntropyLoss()

# Run validation
avg_val_loss, val_solve_rate, val_token_acc = validate_combined(
    model=model,
    schedule=schedule, 
    loader=test_val_loader,
    device=device,
    loss_fn=loss_fn,
    vocab_size=vocab_size
)

print(f"Validation loss: {avg_val_loss:.4f}")
print(f"Solve rate on validation set: {val_solve_rate*100:.2f}%")
print(f"Token-level accuracy on validation set: {val_token_acc*100:.2f}%")